In [1]:
from datasets import load_dataset
from glob import glob
from random import shuffle

train_files = glob("data/train/docs_*.jsonl")
shuffle(train_files)

test_files = glob("data/test/docs_*.jsonl")

data_files = {
    "train": train_files,
    "test": test_files
}

dataset = load_dataset("json", data_files=data_files, streaming=True)
train_dataset = dataset["train"]
test_dataset = dataset["test"]

In [2]:
from utils.streaming_dataset import StreamingTokenDataset
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
# from transformers import AutoModel
# model = AutoModel.from_pretrained("allegro/herbert-base-cased")

tokenizer = AutoTokenizer.from_pretrained("allegro/herbert-base-cased")

CONTEXT_LENGTH = 128
BATCH_SIZE = 32

train_loader = DataLoader(StreamingTokenDataset(train_dataset, tokenizer, context_size=CONTEXT_LENGTH), batch_size=BATCH_SIZE)
test_loader = DataLoader(StreamingTokenDataset(test_dataset, tokenizer, context_size=CONTEXT_LENGTH), batch_size=BATCH_SIZE)


In [3]:
from architectures.lstm import SimpleLSTM

vocab_size = 50_000   # number of tokens
embed_dim = 384     # embedding dimension
hidden_dim = 384     # LSTM hidden size
num_layers = 2

lstm = SimpleLSTM(vocab_size, embed_dim, hidden_dim, num_layers)
param_count = sum(p.numel() for p in lstm.parameters() if p.requires_grad)
print(f"LSTM has {param_count} trainable params")

LSTM has 21615440 trainable params


In [4]:
import torch

def choose_device() -> str:
    if torch.cuda.is_available():
        return "cuda"
    elif torch.backends.mps.is_available():
        return "mps"
    else:
        return "cpu"

In [6]:
train_batch_count = 0
for _ in train_loader:
    train_batch_count += 1

test_batch_count = 0
for _ in test_loader:
    test_batch_count += 1

Token indices sequence length is longer than the specified maximum sequence length for this model (2746 > 512). Running this sequence through the model will result in indexing errors


In [7]:
train_batch_count, test_batch_count

(5421, 56)

In [8]:
import torch.nn as nn
from tqdm import tqdm

epochs = 25
learning_rate = 1e-3
weight_decay = 1e-2
grad_clip = 1.0
device = torch.device(choose_device())

print(f"Training on device: {device}")

lstm.to(device)
criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
optimizer = torch.optim.AdamW(lstm.parameters(), lr=learning_rate, weight_decay=weight_decay)

for epoch in range(epochs):

    lstm.train()
    total_loss = 0.0

    progress = tqdm(enumerate(train_loader), total=train_batch_count, desc=f"Epoch {epoch + 1}/{epochs}")

    for i, (batch_x, batch_y) in progress:
        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device)

        optimizer.zero_grad()
        out, _ = lstm(batch_x)

        # Flatten for CrossEntropyLoss
        loss = criterion(out.view(-1, out.size(-1)), batch_y.view(-1))
        loss.backward()

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(lstm.parameters(), grad_clip)

        optimizer.step()

        total_loss += loss.item()
        avg_loss = total_loss / (i + 1)

        progress.set_postfix({"loss": f"{avg_loss:.4f}", "lr": optimizer.param_groups[0]["lr"]})

    if epoch % 3 == 2:
        torch.save(lstm.state_dict(), f"lstm_epoch_{epoch + 1}.pt")


    print(f"Epoch {epoch} done | Average training loss: {avg_loss:.4f}")
    print(f"Perplexity on training data: {torch.math.exp(avg_loss)}\n")

    with torch.no_grad():
        progress = tqdm(enumerate(test_loader), total=test_batch_count, desc=f"Epoch {epoch + 1}/{epochs}")

        for i, (batch_x, batch_y) in progress:
            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)

            out, _ = lstm(batch_x)
            loss = criterion(out.view(-1, out.size(-1)), batch_y.view(-1))

            total_loss += loss.item()
            avg_loss = total_loss / (i + 1)

    print(f"Average loss on held-out_dataset: {avg_loss:.4f}")
    print(f"Perplexity on held-out data: {torch.math.exp(avg_loss)}\n")

torch.save(lstm.state_dict(), "lstm_final.pt")
print("Training complete. Model saved to lstm_final.pt")


Training on device: cuda


Epoch 1/25:   0%|          | 0/5421 [00:00<?, ?it/s]

Epoch 1/25: 100%|██████████| 5421/5421 [09:36<00:00,  9.41it/s, loss=6.5358, lr=0.001]


Epoch 0 done | Average training loss: 6.5358
Perplexity on training data: 689.35808301941



Epoch 1/25: 100%|██████████| 56/56 [00:02<00:00, 22.89it/s]


...

Average loss on held-out_dataset: 385.0223
Perplexity on held-out data: 1.6333364711154528e+167



Epoch 25/25: 100%|██████████| 5421/5421 [09:39<00:00,  9.35it/s, loss=3.9120, lr=0.001]


Epoch 24 done | Average training loss: 3.9120
Perplexity on training data: 49.99686635682195



Epoch 25/25: 100%|██████████| 56/56 [00:02<00:00, 22.36it/s]


Average loss on held-out_dataset: 383.7878
Perplexity on held-out data: 4.7525825485872903e+166

Training complete. Model saved to lstm_final.pt
